# Group Project Proposal

Group: 83

### Title

### Introduction:
- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question

Accroding to CDC, heart disease is one of the leading cause of death around the world. The most common risk factors that cause heart disease are high blood pressure and cholesterol. That's why we have chosen these two variables as predictors for our data analysis. 

- trestbpd: resting blood pressure
- chol: serum cholesterol in mg/dl

Predictive question: Can we use the blood pressure and cholestoral to predict whether a person has heart disease or not?

This dataset is from the UCI Machine Learning Repository. The repository contains four databases about heart disease from four different places: Cleveland, Budapset, California, and Zurich. We will be using the Cleveland version of the database. 
- To describe the data set put the names of each of the attributes, variable type and a brief description about the attribute.

### Preliminary exploratory data analysis:
- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [29]:
library(tidyverse)

In [32]:
download.file(url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", destfile = "data/processed.cleveland.data")
data <- read_csv(file = "data/processed.cleveland.data", col_names = c("age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num_the_predicted_attribute"))

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Because it says experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0), for this project, value 1,2,3,4 have the same meaning, so I make value 2,3,4 in num_the_predicted_attribute to be value 1.

In [33]:
data1 <- filter(data, num_the_predicted_attribute <=1)
data2 <- filter(data, num_the_predicted_attribute > 1)
data2 <- mutate(data2, num_the_predicted_attribute = 1)
data <- merge(data1, data2, all = T)
data

age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num_the_predicted_attribute
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
29,1,2,130,204,0,2,202,0,0.0,1,0.0,3.0,0
34,0,2,118,210,0,0,192,0,0.7,1,0.0,3.0,0
34,1,1,118,182,0,2,174,0,0.0,1,0.0,3.0,0
35,0,4,138,183,0,0,182,0,1.4,1,0.0,3.0,0
35,1,2,122,192,0,0,174,0,0.0,1,0.0,3.0,0
35,1,4,120,198,0,0,130,1,1.6,2,0.0,7.0,1
35,1,4,126,282,0,2,156,1,0.0,1,0.0,7.0,1
37,0,3,120,215,0,0,170,0,0.0,1,0.0,3.0,0
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0


### Methods:
- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results

### Expected outcomes and significance:
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?